In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow-gpu==2.0.0

In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers
print(tf.__version__)
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
import itertools
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from sklearn.metrics import confusion_matrix


2.3.0


In [5]:
size = 25

In [6]:
'''config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)'''

'config = tf.compat.v1.ConfigProto()\nconfig.gpu_options.allow_growth = True\nsess = tf.compat.v1.Session(config=config)\nconfig = tf.compat.v1.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.95\ntf.compat.v1.Session(config=config)'

![title](擷取.PNG)

In [7]:
def gernate_data(n, nu, seed, range_, multi, size):
    ## normal
    np.random.seed(seed=seed)
    y_train = np.ones((int(n*(1-nu)),))
    mean = np.random.uniform(-5, 5, size)
    cov_  = np.random.uniform(0, 1, (size, size))* 3
    cov = np.dot(cov_.T,cov_)
    x_train = np.random.multivariate_normal(mean, cov, (int(n*(1-nu))))
    x_train = x_train.astype('float32')

    
    
    np.random.seed(seed=seed)
    x_trains_ = None ## normal + outlier
    for i in range(multi):
        mean = np.random.uniform(-range_, range_, size)
        cov_ = np.random.randn(size, size)
        cov = np.dot(cov_.T,cov_)
        x_train_ = np.random.multivariate_normal(mean, cov, (int(n*nu/multi)), check_valid  = 'raise')
        try:
            x_trains_ = np.vstack((x_trains_, x_train_))
        except:
            x_trains_ = x_train_
    ## 畫圖
    '''fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train[:, 0], x_train[:, 1],x_train[:, 2], s=0.9, label = 'normal')
    ax.scatter(x_trains_[:,0], x_trains_[:,1], x_trains_[:,2], s=0.9, label = 'outlier')
    plt.legend()
    plt.title('normal + outlier')
    plt.show()'''

    y_trains_ = np.zeros(((len(x_trains_),)))
    x_train_mix = np.vstack((x_trains_, x_train)).astype('float32')
    y_train_mix = np.hstack((y_trains_, y_train)).astype('float32')
    
    return x_train_mix, y_train_mix

## 參數設定

In [8]:

z_dim = 5
h_dim = [10, 5]
regular = 1e-5
max_lr = 1e-6
base_lr = 1e-5


n_epochs = 1000

optimizer = tf.keras.optimizers.Adam(lr = base_lr)

In [9]:
def make_ae_model():

    inputs = tf.keras.Input(shape=(size,), name='inputs')

    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(inputs)
    x = tf.keras.layers.Dense(h_dim[1], activation='linear', use_bias=False, name = 'hidden')(x)
    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(x)
    x = tf.keras.layers.Dense(size, activation='linear', use_bias=False)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

ae_model = make_ae_model()
ae_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 25)                250       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [10]:
def make_model(model):
    
    model.trainable = True
    x = model.get_layer('hidden').output

    model_out = tf.keras.Model(inputs = model.input, outputs = x)
    
    return model_out

model = make_model(ae_model)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
Total params: 300
Trainable params: 300
Non-trainable params: 0
_________________________________________________________________


## Train SA-SVDD

In [20]:
class SA_DeepSVDD_Trainer():
    
    def __init__(self, nu, SVDD_model, train_dataset, warm_up_n_epochs):
        self.SVDD_model = SVDD_model
        self.nu = nu
        self.train_dataset = train_dataset
        self.warm_up_n_epochs = warm_up_n_epochs
        self.nu = nu
        ## 設置lagrange因子、Radius
        self.center = None
        self.optimizer = tf.keras.optimizers.Adam(lr = 1e-3, epsilon = 1e-6)
        
    def init_center_c(self, x_train_mix):

        modeloutput = self.SVDD_model(x_train_mix, training=False)            
        c = tf.Variable(tf.reduce_mean(modeloutput, axis = 0, name = 'outputs')) 
        
        return c
    def get_radius(self, data):
        
        output = self.SVDD_model(data, training=False)  
        distance = output - self.center
        dis_square = tf.reduce_sum(tf.square(distance), 1)
        
        return tfp.stats.percentile(tf.sqrt(dis_square), (1 - self.nu)*100)
   
    def loss_function(self, output, center):
        
        distance = output - center 
        all_distance = tf.reduce_sum(tf.square(distance), 1)
        objective =  tf.reduce_mean(all_distance)
        
        return objective
    
    @tf.function
    def train_model(self, data, center):
        with tf.GradientTape() as tape:
            output = self.SVDD_model(data, training=True)
            loss_train = self.loss_function( output, center)

        grads = tape.gradient(loss_train, ( self.SVDD_model.trainable_variables))
        self.optimizer.apply_gradients(zip(grads, (self.SVDD_model.trainable_variables)))

        return loss_train
    
    @tf.function
    def train_center(self, data, center):
        with tf.GradientTape() as tape:
            output = self.SVDD_model(data, training=False)
            loss_train = self.loss_function( output, center)

        grads = tape.gradient(loss_train, ([self.center]))
        self.optimizer.apply_gradients(zip(grads, ([self.center])))

        return loss_train
    
    def train(self, x_train_mix):
        objective_break = False
   
        objective_list = []
        validation_loss_list = []

        
        pre_objective_loss = np.inf
        objective_loss = 0
        
        
        epoch = 0
        save = False
        self.center = self.init_center_c(x_train_mix)
        #for epoch in range(200):
        while abs(pre_objective_loss - objective_loss) > 1e-6  :
            epoch_loss_avg = tf.metrics.Mean()
            if epoch > self.warm_up_n_epochs:
                pre_objective_loss = objective_loss
           
            for batch, (batch_x) in enumerate(self.train_dataset):
                loss = self.train_model(batch_x,  self.center)
                epoch_loss_avg(loss)
                if (epoch >= self.warm_up_n_epochs):
                    self.train_center(batch_x, self.center)
                else:
                    self.center.assign(self.init_center_c(x_train_mix))
                
            objective_loss = epoch_loss_avg.result()
            #objective_list.append(loss)
            epoch += 1
        #plt.plot(objective_list)
        #plt.show()

        #print('{:4d}: R: {:.4f} objective loss:{:4f}  lr:{:4f}'.format(
            #epoch, self.radius.numpy(), objective_loss,  self.optimizer.lr.numpy()))
        ### 決定使用的model

            
        self.SVDD_model.save_weights('model')
        np.save('center' , self.center.numpy())
        radius = self.get_radius(x_train_mix)
        np.save('radius' , radius.numpy())

    def distance(self, data, load_model = True):
        if load_model:
            self.SVDD_model.load_weights('model')
            center_end = np.load('center.npy')
            radius_end =  np.load('radius.npy')

        output = self.SVDD_model(tf.convert_to_tensor(data), training=False)
        distance = output - center_end
        dis_square = tf.reduce_sum(tf.square(distance), 1)                 
        dis_square = dis_square.numpy()
        
        return dis_square, radius_end
    
    def result_plot (self, dists, x_train_mix, radius ):
        r = radius**2
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        plt.show()
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        ax.scatter(x_train_mix[dists>=r][:,0], x_train_mix[dists>=r][:,1],x_train_mix[dists>=r][:,2] ,s=0.9, label='anomaly')
        plt.legend()
        plt.show()
    
    def train_result(self, x_train_mix, show_img = False, load_model = True):

        if show_img:
            dis_square, radius_end = self.distance(x_train_mix, load_model = load_model)
            #self.result_plot (dis_square, x_train_mix, radius_end )

        return radius_end
    
    def result(self, dist, y_test, radius_end, show_img = False):
        
        y_true = y_test.astype(int)
        scores = dist - radius_end ** 2
        y_prob = 1/dist
        auc = roc_auc_score(y_true, y_prob)
        confusionmatrix = confusion_matrix(y_true, scores<0)
        if show_img:
            print('AUC:', auc)
            print('confusion_matrix:')
            print(confusionmatrix)

        return auc, confusionmatrix 
    
    def test_result(self, x_test_mix, y_test_mix, show_img = False, load_model = True):
        
        dis_square, radius_end = self.distance(x_test_mix, load_model = load_model)
        dis_square += 1e-16
        test_auc, confusionmatrix = self.result(dis_square, y_test_mix, radius_end, show_img = show_img)
        if show_img :
            self.result_plot (dis_square, x_test_mix, radius_end )
        

        return test_auc, confusionmatrix

In [28]:
nu = 0.1
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [29]:
result = np.zeros((25, 3))

In [30]:
warm_up_n_epochs = 100
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 5, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, train_dataset, warm_up_n_epochs)
    SVDD.train(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9092857463524132
epoch =  1 auc =  0.9478347923681258
epoch =  2 auc =  0.7881341189674523
epoch =  3 auc =  0.8963590347923681
epoch =  4 auc =  0.8055592592592592
epoch =  5 auc =  0.8638607182940516
epoch =  6 auc =  0.8876902356902358
epoch =  7 auc =  0.9385928170594836
epoch =  8 auc =  0.8862140291806959
epoch =  9 auc =  0.8925386083052749
epoch =  10 auc =  0.7905167227833894
epoch =  11 auc =  0.8973668911335579
epoch =  12 auc =  0.898872278338945
epoch =  13 auc =  0.8858991021324354
epoch =  14 auc =  0.9712398428731761
epoch =  15 auc =  0.8974457912457912
epoch =  16 auc =  0.9561736251402918
epoch =  17 auc =  0.9097255892255893
epoch =  18 auc =  0.842421997755331
epoch =  19 auc =  0.9309754208754208
epoch =  20 auc =  0.8817406285072952
epoch =  21 auc =  0.9160243546576881
epoch =  22 auc =  0.8917068462401797
epoch =  23 auc =  0.8918102132435466
epoch =  24 auc =  0.9183286195286195


In [31]:
np.mean(result, 0)

array([12.        ,  0.89185269, 55.36153605])

In [32]:
np.std(result, 0)

array([ 7.21110255,  0.04526187, 11.20951504])

In [35]:
nu = 0.2
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [36]:
result = np.zeros((25, 3))
warm_up_n_epochs = 100
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 5, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, train_dataset, warm_up_n_epochs)
    SVDD.train(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.8431755012531329
epoch =  1 auc =  0.8765227443609023
epoch =  2 auc =  0.9293547619047617
epoch =  3 auc =  0.8992200501253134
epoch =  4 auc =  0.9344097744360902
epoch =  5 auc =  0.8694468984962406
epoch =  6 auc =  0.8608295739348371
epoch =  7 auc =  0.8946831453634084
epoch =  8 auc =  0.7663335839598997
epoch =  9 auc =  0.8173206766917294
epoch =  10 auc =  0.8970941102756892
epoch =  11 auc =  0.9444944235588971
epoch =  12 auc =  0.7607317042606517
epoch =  13 auc =  0.9024353383458646
epoch =  14 auc =  0.879696679197995
epoch =  15 auc =  0.8965323934837094
epoch =  16 auc =  0.8891835213032582
epoch =  17 auc =  0.9460774436090226
epoch =  18 auc =  0.9392875
epoch =  19 auc =  0.9109057644110276
epoch =  20 auc =  0.8230522556390977
epoch =  21 auc =  0.9407681077694234
epoch =  22 auc =  0.9525062656641605
epoch =  23 auc =  0.9398120300751881
epoch =  24 auc =  0.9155600877192982


In [37]:
np.mean(result, 0)

array([12.        ,  0.88917737, 46.70352222])

In [38]:
np.std(result, 0)

array([ 7.21110255,  0.05214618, 10.22475828])

In [39]:
nu = 0.3
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [40]:
result = np.zeros((25, 3))
warm_up_n_epochs = 100
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 5, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, train_dataset, warm_up_n_epochs)
    SVDD.train(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.7974511190476191
epoch =  1 auc =  0.8926670476190475
epoch =  2 auc =  0.8060963333333334
epoch =  3 auc =  0.8278205238095238
epoch =  4 auc =  0.7808373571428572
epoch =  5 auc =  0.8847112857142857
epoch =  6 auc =  0.7407251904761905
epoch =  7 auc =  0.9514927619047621
epoch =  8 auc =  0.8461288095238094
epoch =  9 auc =  0.9577614047619047
epoch =  10 auc =  0.8772501904761905
epoch =  11 auc =  0.893010619047619
epoch =  12 auc =  0.8474437142857143
epoch =  13 auc =  0.9068737619047618
epoch =  14 auc =  0.8805217142857144
epoch =  15 auc =  0.8548119523809523
epoch =  16 auc =  0.9005858095238094
epoch =  17 auc =  0.8939415714285713
epoch =  18 auc =  0.7272076666666667
epoch =  19 auc =  0.7558107857142857
epoch =  20 auc =  0.8648383333333333
epoch =  21 auc =  0.8473114761904761
epoch =  22 auc =  0.9204619999999999
epoch =  23 auc =  0.9525552380952382
epoch =  24 auc =  0.885109761904762


In [41]:
np.mean(result, 0)

array([12.        ,  0.85973706, 39.23999285])

In [42]:
np.std(result, 0)

array([7.21110255, 0.06218005, 7.43863186])